In [2]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
#import smogn
import seaborn as sns
import pickle

from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import math

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer 
from imblearn.over_sampling import SMOTE
from sklearn.metrics import matthews_corrcoef
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
#from tensorflow.keras.models import model_from_json

#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')

#!python -m spacy download en_core_web_sm
nlp_sm = spacy.load('en_core_web_sm')

#import en_core_web_sm
#nlp_md = en_core_web_sm.load()y

#import en_core_web_md
#nlp_md = en_core_web_md.load()
#!python -m spacy download en_core_web_md
nlp_md = spacy.load('en_core_web_md')

import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"


In [4]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
from src import ensemble_tree_models
from src import tunning_hyperparametrs as th
#from src import mlp_regressor
#from src import mlp_classifier
from src import summarization
from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data
from src import display_results as dr
import joblib
from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
from src import pipeline_extract_features as pef

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def main(path_to_read, name_csv="features", format_dataset=True, verbose=True):
    
    columns_name = ['text_rank', 'lex_rank', 'count_one_gram',
        'count_article_keywords', 'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

    sections=['introduction', 'materials', 'conclusion', 'concat']
    #sections=[ 'introduction']

    if format_dataset:
        if verbose:
            print("Preparando dataset para os classificadores")
        dataset = prepare_data.main_create_dataset(columns_name, sections, path_to_read, name_csv)
        #utils.save_json(dataset, name='dataset', path=path)
    else:
        if verbose:
            print("Carregando dataset")
        dataset = utils.load_json(name='dataset', path=path)
    
    if verbose:
        print("Treinamento dos modelos")
        
    with open('dataset_{}.pkl'.format(name_csv), 'wb') as fp:
        pickle.dump(dataset, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
    return dataset
    
    '''models = ensemble_tree_models.create_models(dataset, sections, name_models=['knn', 'gb', 'rf', 'ab'])
    predictions, results = ev.create_reports(models, dataset, columns_name, verbose=False)
    
    #utils.save_json(predictions, name='prediction', path=path)
    #utils.save_results(results, path=path)
    
    parameters = {'introduction': [0.2, 100, 64],
             'materials': [0.2, 100, 64],
             'conclusion':[0.2, 100, 64],
             'concat': [0.2, 100, 64]}

    models_nn = mlp_classifiers.main_train_nn(dataset, sections, parameters, train=True, verbose=False)
    predictions, results =  mlp_classifiers.eval_nn(dataset, sections)'''
    

In [38]:
path_to_read="/scratch/cinthiasouza/mv-text-summarizer/result_/{}/{}_*.csv"

In [ ]:
dataset = main(verbose=True, path_to_read=path_to_read, name_csv="embeddings")

Preparando dataset para os classificadores
/scratch/cinthiasouza/mv-text-summarizer/result_/introduction/embeddings_*.csv


In [7]:
features_intro

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,article
0,0.019369,0.170023,-0.157492,0.084929,0.044337,-0.125541,-0.005107,-0.066788,0.119723,1.854209,...,0.086841,0.006963,-0.256000,-0.088001,0.057092,-0.068424,-0.018214,-0.042552,-0.000397,PMC3291329.json
1,-0.109538,0.153488,-0.091134,-0.045741,0.031879,-0.065606,-0.003806,0.007853,0.086485,1.895246,...,0.090416,-0.074388,-0.266435,-0.140531,0.212975,-0.066952,-0.088597,-0.032297,-0.063292,PMC3291329.json
2,-0.230669,0.239444,-0.050260,-0.046963,0.041958,-0.173455,0.057514,0.026981,0.132307,1.829763,...,-0.010914,-0.097157,-0.129535,-0.059470,0.088565,-0.119766,-0.090443,-0.021378,-0.057760,PMC3291329.json
3,0.028923,0.104067,-0.036298,-0.080122,-0.066425,-0.050970,0.076039,-0.102261,0.113465,2.123320,...,0.123431,-0.102774,-0.178460,0.024490,0.053271,-0.170324,0.004651,-0.043970,-0.081540,PMC3291329.json
4,-0.234796,0.239623,-0.232185,-0.025667,-0.173410,-0.087379,0.001200,0.045584,0.075028,1.994535,...,0.137531,-0.107279,-0.164704,-0.041849,0.026744,-0.060013,-0.071192,0.069059,0.060591,PMC3291329.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251312,-0.173795,0.066398,-0.116225,0.017819,0.003540,-0.017307,-0.062365,-0.003871,0.066472,1.401559,...,0.121018,0.031351,-0.230207,-0.081620,-0.037723,-0.060129,-0.000553,-0.049476,0.184041,PMC3306492.json
251313,-0.049494,0.063398,-0.078736,0.078287,-0.048849,-0.115554,-0.049605,0.028073,0.106532,1.767797,...,0.084401,0.129419,-0.157510,-0.071702,0.042163,-0.008709,0.073630,0.027167,0.173512,PMC3306492.json
251314,-0.043679,0.042419,-0.214362,-0.026043,-0.078830,-0.010802,0.054261,-0.029083,0.095917,1.722176,...,0.077229,0.088890,-0.116238,-0.058489,0.024350,-0.124215,0.121618,0.065002,0.024998,PMC3306492.json
251315,-0.109918,0.178515,-0.180347,0.064858,-0.063268,-0.241080,-0.005463,0.029733,0.259321,1.918528,...,0.044287,0.085623,-0.163573,-0.004580,-0.049780,-0.047489,0.157098,0.050879,0.085770,PMC3306492.json
